In [135]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from textblob import TextBlob
import numpy as np
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from wordcloud import WordCloud
from pprint import pprint
import os
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

stop_words = nltk.corpus.stopwords.words('english')

tweets = pd.read_csv(r'/content/drive/MyDrive/UT Austin/College/Fall/Unstructured/Project/tweets_final.csv')
tweets['clean_text'] = np.nan

remove_rt = lambda x: re.sub("RT @\w+: "," ",x)
rt = lambda x: ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", x).split())
tweets['clean_text'] = tweets.text.map(remove_rt).map(rt)
tweets['clean_text'] = tweets.clean_text.str.lower()

tweets[['polarity', 'subjectivity']] = tweets['clean_text'].apply(lambda x: pd.Series(TextBlob(x).sentiment))
tweets['sentiment'] = np.nan
tweets['neg'] = np.nan
tweets['neu'] = np.nan
tweets['pos'] = np.nan
tweets['compound'] = np.nan

for index, row in tweets['clean_text'].iteritems():
 score = SentimentIntensityAnalyzer().polarity_scores(row)
 neg = score['neg']
 neu = score['neu']
 pos = score['pos']
 comp = score['compound']
 if neg > pos:
  tweets.loc[index, 'sentiment'] = 'negative'
 elif pos > neg:
  tweets.loc[index, 'sentiment'] = 'positive'
 else:
  tweets.loc[index, 'sentiment'] = 'neutral'
 tweets.loc[index, 'neg'] = neg
 tweets.loc[index, 'neu'] = neu
 tweets.loc[index, 'pos'] = pos
 tweets.loc[index, 'compound'] = comp

<input>:22: DeprecationWarning: invalid escape sequence \w
<input>:22: DeprecationWarning: invalid escape sequence \w
<input>:23: DeprecationWarning: invalid escape sequence \w
<input>:22: DeprecationWarning: invalid escape sequence \w
<input>:23: DeprecationWarning: invalid escape sequence \w
<input>:22: DeprecationWarning: invalid escape sequence \w
<input>:23: DeprecationWarning: invalid escape sequence \w
<input>:22: DeprecationWarning: invalid escape sequence \w
<input>:23: DeprecationWarning: invalid escape sequence \w
<input>:22: DeprecationWarning: invalid escape sequence \w
<input>:23: DeprecationWarning: invalid escape sequence \w
<input>:22: DeprecationWarning: invalid escape sequence \w
<input>:23: DeprecationWarning: invalid escape sequence \w
<input>:22: DeprecationWarning: invalid escape sequence \w
<input>:23: DeprecationWarning: invalid escape sequence \w
<input>:22: DeprecationWarning: invalid escape sequence \w
<input>:23: DeprecationWarning: invalid escape sequence 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [136]:
tweets.to_csv(r'/content/drive/MyDrive/UT Austin/College/Fall/Unstructured/Project/tweets_final_sentiment.csv')

tweets_pos = tweets[tweets['sentiment']=='positive']
tweets_neg = tweets[tweets['sentiment']=='negative']
tweets_neu = tweets[tweets['sentiment']=='neutral']

print(tweets_pos)
print(tweets_neg)
print(tweets_neu)

            user_id             user      tweet_id  ...    neu    pos compound
5      2.337379e+08    SenRonJohnson  1.445120e+18  ...  0.845  0.155   0.2960
17     1.489784e+07          7NewsDC  1.445120e+18  ...  0.929  0.071   0.0772
19     2.433536e+09    stellaaa_wood  1.445120e+18  ...  0.652  0.187   0.3810
20     1.263180e+18     KevinCChiang  1.445120e+18  ...  0.627  0.277   0.5994
36     1.379180e+18  organichockeyg2  1.445120e+18  ...  0.864  0.136   0.2960
...             ...              ...           ...  ...    ...    ...      ...
60084  1.202287e+07   philmichaelson  1.442960e+18  ...  0.793  0.115   0.1280
60085  1.737968e+07            NBCLA  1.442960e+18  ...  0.896  0.104   0.1027
60086  2.730345e+09         dhfull50  1.442960e+18  ...  0.657  0.221   0.4404
60092  2.608730e+09       CermwgpSue  1.442960e+18  ...  0.766  0.234   0.6369
60093  8.265260e+17      Chrisko6379  1.442960e+18  ...  0.754  0.126   0.0258

[19655 rows x 14 columns]
            user_id      

In [137]:

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def topic_model(tweets_var):
    tweet_sentences = tweets_var['clean_text'].values.tolist()
    tweet_words = list(sent_to_words(tweet_sentences))
    # remove stop words
    tweet_words = remove_stopwords(tweet_words)


    # Create Dictionary
    id2word = corpora.Dictionary(tweet_words)
    # Create Corpus
    texts = tweet_words
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    # number of topics
    num_topics = 15
    # Build LDA model
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
    doc_lda = lda_model[corpus]
    return doc_lda, lda_model

In [138]:
doc_lda_pos, lda_model_pos = topic_model(tweets_pos)
doc_lda_neg, lda_model_neg = topic_model(tweets_neg)
doc_lda_neu, lda_model_neu = topic_model(tweets_neu)



Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [139]:
def strip_string(string):
  string = string.lstrip("' ")
  string = string.rstrip("\" ")
  string = string.split("*\"")
  return string

def topics_populate(name,topics_df,lda):
  for i in range(lda.num_topics):
    list_terms = list(map(strip_string,lda.print_topics()[i][1].split('+')))
    for j in range(len(list_terms)):
      if list_terms[j][1] not in topics_df.index.tolist():
        new_row = np.zeros(lda.num_topics)
        new_row[i] = list_terms[j][0]
        new_row = pd.Series(new_row)
        new_row.name = list_terms[j][1]
        topics_df = topics_df.append(new_row)
      else:
        topics_df.loc[list_terms[j][1],i] = list_terms[j][0]

  topics_df = topics_df.drop(['vaccine','covid'])
  topics_df.to_csv(r'/content/drive/MyDrive/UT Austin/College/Fall/Unstructured/Project/{0}.csv'.format(name))

In [140]:
topics_pos = pd.DataFrame()
topics_neg = pd.DataFrame()
topics_neu = pd.DataFrame()

topics_populate('topics_pos',topics_pos,lda_model_pos)
topics_populate('topics_neg',topics_neg,lda_model_neg)
topics_populate('topics_neu',topics_neu,lda_model_neu)

In [146]:
def add_topics_to_tweets(name,topics_df,tweets_var,doc):
  topic_scores = [[topic_score[1] for topic_score in tweet] for tweet in doc]
  df_topics = pd.DataFrame(topic_scores)
  max_topics = [max(tweet, key=lambda x: x[1])[0] for tweet in doc]
  max_topics_df = pd.DataFrame(max_topics)

  tweets_var.reset_index(inplace=True)
  tweets_var.drop('index',axis=1,inplace=True)
  tweets_var['topic_id'] = max_topics_df

  topics_df = pd.read_csv(r'/content/drive/MyDrive/UT Austin/College/Fall/Unstructured/Project/{0}.csv'.format(name))
  topics_df.set_index('Unnamed: 0',inplace=True)
  topics_df = topics_df.columns.to_frame().reset_index()

  tweets = tweets_var.join(topics_df[0], how='left',on='topic_id')
  tweets.rename({0:'topic'},axis=1,inplace=True)
  tweets.to_csv(r'/content/drive/MyDrive/UT Austin/College/Fall/Unstructured/Project/{0}.csv'.format(name))

In [147]:
add_topics_to_tweets('topics_pos',topics_pos,tweets_pos,doc_lda_pos)
add_topics_to_tweets('topics_neg',topics_neg,tweets_neg,doc_lda_neg)
add_topics_to_tweets('topics_neu',topics_neu,tweets_neu,doc_lda_neu)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  2  3   6   9 -3
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [122]:
#tweets_pos.reset_index(inplace=True)
#tweets_pos.drop('index',axis=1,inplace=True)
#tweets_pos['topic_id'] = max_topics_df

topics_pos = pd.read_csv(r'/content/drive/MyDrive/UT Austin/College/Fall/Unstructured/Project/topics_pos.csv')
topics_pos.set_index('Unnamed: 0',inplace=True)
topics_pos = topics_pos.columns.to_frame().reset_index()

tweets = tweets_pos.join(topics_pos[0], how='left',on='topic_id')
tweets.rename({0:'topic'},axis=1,inplace=True)
tweets.to_csv(r'/content/drive/MyDrive/UT Austin/College/Fall/Unstructured/Project/tweets_pos.csv')


user_id             user  ...  topic_id        topic
0      2.337379e+08    SenRonJohnson  ...         5    Long Term
1      1.489784e+07          7NewsDC  ...        10     Students
2      2.433536e+09    stellaaa_wood  ...        14      Workers
3      1.263180e+18     KevinCChiang  ...         9  Affirmative
4      1.379180e+18  organichockeyg2  ...         2    Pregnancy
...             ...              ...  ...       ...          ...
19650  1.202287e+07   philmichaelson  ...        10     Students
19651  1.737968e+07            NBCLA  ...        14      Workers
19652  2.730345e+09         dhfull50  ...         2    Pregnancy
19653  2.608730e+09       CermwgpSue  ...         3        Young
19654  8.265260e+17      Chrisko6379  ...         0      Booster

[19655 rows x 17 columns]